In [6]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import *
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as f
import time

In [7]:
sc = SparkContext('local')

In [14]:
from __future__ import print_function
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)

filestream = ssc.textFileStream("/home/spark/ch06input")



<bound method DStream.context of <pyspark.streaming.dstream.DStream object at 0x0000018D45A9AAC0>>

In [9]:
from datetime import datetime
def parseOrder(line):
  s = line.split(",")
  try:
      if s[6] != "B" and s[6] != "S":
        raise Exception('Wrong format')
      return [{"time": datetime.strptime(s[0], "%Y-%m-%d %H:%M:%S"), "orderId": long(s[1]), "clientId": long(s[2]), "symbol": s[3],
      "amount": int(s[4]), "price": float(s[5]), "buy": s[6] == "B"}]
  except Exception as err:
      print("Wrong line format (%s): " % line)
      return []

orders = filestream.flatMap(parseOrder)

In [10]:
from operator import add
numPerType = orders.map(lambda o: (o['buy'], 1L)).reduceByKey(add)

numPerType.repartition(1).saveAsTextFiles("/home/spark/ch06output/output", "txt")

ssc.start()

ssc.stop(False)

allCounts = sc.textFile("/home/spark/ch06output/output*.txt")

SyntaxError: invalid syntax (<ipython-input-10-452bc79e651c>, line 2)